We intend to be able to use around 10,000 labeled data points.

Each data point should incorporate a weather data matrix and a label, like the following

( [45, 345, 2342, 34, 231, ..., 233] , 0 )

We will feed this into the model we set up

In [0]:
from google.colab import auth
auth.authenticate_user()
import gspread
import numpy as np
from oauth2client.client import GoogleCredentials

from IPython.display import display
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
#User enters name of the formatted spreadsheet 
#don't use the file type e.g. file.csv... you'll get errors
worksheet = gc.open('aggregate_features').sheet1
rows = worksheet.get_all_values()[1:]
matrix = np.array(rows)
np.random.shuffle(matrix)
x_all = np.array(matrix[1:,5:],dtype=float)
y_all = np.array(matrix[1:,0],dtype=float)

y_pos = np.where(y_all == 1, True, False)

print(y_pos)

x_pos = x_all[y_all == 1]
y_pos = y_all[y_all == 1]

## normalize
normalization_factors = [15.33742331,21.23142251,0.01448435689,0.01399972001,0.0149298298,0.01936108422,0.01880759827,0.01802776275,1.041666667,1.030927835,1.041666667,1,0.04321521175]
normalization_constants = [0,0,-0.6519409038,-0.564748705,-0.6549716333,-0.5537270087,-0.4920067707,-0.5238867856,-0.04166666667,-0.03092783505,-0.04166666667,0,-0.03975799481]
x_all = np.multiply(normalization_factors, x_all) + normalization_constants
x_pos = np.multiply(normalization_factors, x_pos) + normalization_constants
print(x_all.shape)
print(x_all)
print(y_all.shape)
print(y_all)
print(x_pos.shape)
print(x_pos)
print(y_pos.shape)
print(y_pos)



[ True False  True ... False False False]
(3136, 13)
[[0.02453988 0.01698514 0.56662804 ... 0.48958333 0.04       0.10803803]
 [0.0506135  0.03397028 0.86051564 ... 0.04166667 0.         0.21910112]
 [0.0398773  0.02760085 0.72392816 ... 0.28125    0.05       0.20354365]
 ...
 [0.         0.         0.49391657 ... 0.44791667 0.         0.17372515]
 [0.06288344 0.02972399 0.46205098 ... 0.5        0.01       0.14649957]
 [0.0506135  0.02547771 0.70321553 ... 0.25       0.         0.13094209]]
(3136,)
[1. 0. 1. ... 0. 0. 0.]
(905, 13)
[[0.02453988 0.01698514 0.56662804 ... 0.48958333 0.04       0.10803803]
 [0.0398773  0.02760085 0.72392816 ... 0.28125    0.05       0.20354365]
 [0.02760736 0.014862   0.72059676 ... 0.25       0.1        0.1119274 ]
 ...
 [0.04907975 0.04670913 0.56199305 ... 0.19791667 0.02       0.12878133]
 [0.05521472 0.02123142 0.8716686  ... 0.44791667 0.         0.46110631]
 [0.         0.         0.41990151 ... 0.55208333 0.73       0.10630942]]
(905,)
[1. 1. 1. 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from sklearn.model_selection import KFold


ns= 10
resultsArr = []
resultsArr2 = []



# kfold + shuffle
kfs = KFold(n_splits=ns, shuffle=True)

model = Sequential()
print(len(x_all[0]))
print(x_all[0])
model.add(Dense(len(x_all[0]), input_dim=len(x_all[0])))
model.add(Activation('sigmoid'))
model.add(Dense(int(len(x_all[0])//2)))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

for layer in model.layers:
    print(layer.input_shape)
    print(layer.output_shape)

model.compile(optimizer='sgd',
                          loss='MSE',
                          metrics=['accuracy'])
#or just 'sgd'

mapdata = []
# train
for train_index, test_index in kfs.split(x_all):
        print(train_index)
        print(test_index)
        x_train, x_test = x_all[train_index], x_all[test_index]
        y_train, y_test = y_all[train_index], y_all[test_index]

        model.fit(x=x_train, y=y_train, epochs=4, batch_size=4)
        preds = model.predict(x_test)
        results = model.evaluate(x_test, y_test, batch_size=4)
        results2 = model.evaluate(x_pos, y_pos, batch_size=4)
        print(results)
        print("Based results: " + str(results2))
        fnr = 0
        fpr = 0
        tpr = 0
        tnr = 0
        #print(preds)
        for p in range(len(preds)):
          print(preds[p], end=', ')
          print(y_test[p])
          if round(float(preds[p])) == 0 and y_test[p] == 1:
            fnr += 1
          if round(float(preds[p])) == 1 and y_test[p] == 0:
            fpr += 1
          if round(float(preds[p])) == 1 and y_test[p] == 1:
            tpr += 1
          if round(float(preds[p])) == 0 and y_test[p] == 0:
            tnr += 1
        fnr = fnr/len(preds)
        fpr = fpr/len(preds)
        tpr = tpr/len(preds)
        tnr = tnr/len(preds)
        print("\nFalse negative rate: " + str(fnr)) 
        print("\nFalse positive rate: " + str(fpr)) 
        print("\nTrue positive rate: " + str(tpr)) 
        print("\nTrue negative rate: " + str(tnr)) 
        resultsArr.append(results)
        resultsArr2.append(results2)
        mapdata.append(model.predict(x_map))

print(max(resultsArr, key=lambda x:x[1]))
print(max(resultsArr2, key=lambda x:x[1]))
for m in range(len(mapdata)):
  print(mapdata[m], xmap[m])
  


13
[0.05521472 0.02972399 0.47204519 0.52596948 0.40967453 0.46911907
 0.57175099 0.41590049 0.67708333 0.73195876 0.80208333 0.07
 0.20095073]
(None, 13)
(None, 13)
(None, 13)
(None, 13)
(None, 13)
(None, 6)
(None, 6)
(None, 6)
(None, 6)
(None, 1)
(None, 1)
(None, 1)
[   0    1    2 ... 3133 3134 3135]
[   6   10   11   15   38   49   54   57   63   74   76   82   90   91
   99  106  120  121  136  148  149  189  235  240  242  249  260  273
  284  293  303  309  334  349  361  363  368  369  413  414  417  419
  424  434  445  456  463  465  466  468  479  481  487  503  505  513
  515  519  525  540  560  563  564  576  580  601  616  620  627  630
  642  643  652  653  662  684  686  705  717  722  730  739  754  755
  767  773  776  780  804  808  813  835  849  865  866  886  887  895
  897  903  915  917  925  955  987  996 1003 1008 1025 1032 1041 1068
 1086 1096 1101 1109 1112 1124 1127 1134 1135 1154 1161 1191 1196 1221
 1234 1245 1258 1260 1283 1311 1314 1316 1326 1331 1339 

In [0]:
(x_train, y_train), (x_test, y_test)



#TODO:
#shuffle data: pandas -> numpy
#10-fold cross-validation: numpy -> SKlearn

x_all = np.array([[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], 
                  [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], 
                  [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], 
                  [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], 
                  [1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0]])
print(type(x_all))
print(x_all.shape)
y_all = np.array([1, 1, 0, 1, 1, 0, 
                  1, 1, 0, 1, 1, 0, 
                  1, 1, 0, 1, 1, 0,
                  1, 1, 0, 1, 1, 0,
                  1, 1, 0, 1, 1, 0]).reshape(30,1)

model = Sequential()
model.add(Dense(len(x_train[0]), input_dim=len(x_train[0])))
model.add(Activation('tanh'))
model.add(Dense(len(x_train[0]) * (3//4)))
model.add(Activation('tanh'))
model.add(Dense(len(x_train[0]) * (3//4)))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('relu'))
#or just 'sgd'
model.compile(optimizer=keras.optimizers.SGD(lr=0.01, nesterov=True),
                  loss='MeanSquaredError',
                  metrics=['accuracy'])

one_hot_labels = keras.utils.to_categorical(y_train, num_classes=1)
model.fit(x_train, one_hot_labels, epochs=10, batch_size=len(x_train[0]))
one_hot_test_labels = keras.utils.to_categorical(y_test, num_classes=1)
results = model.evaluate(x_test, one_hot_test_labels, batch_size=32)
  



# normalize if necessary
#x_train, x_test = x_train , x_test / 255.0

In [0]:
## to create the visualization map
worksheet = gc.open('map data').sheet1
rows = worksheet.get_all_values()[1:]
matrix = np.array(rows)
np.random.shuffle(matrix)
x_map = np.array(matrix[1:,5:],dtype=float)

x_map = np.multiply(x_map, normalization_factors) + normalization_constants
print(x_map)

SpreadsheetNotFound: ignored